In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from tensorflow.python.data import Dataset
from IPython import display

In [2]:
data = pd.read_csv('ted.csv')
data = data.sample(frac=1)

In [5]:
data.shape
#2550개 중, 1600 / 550 / 400

#duration

#comments

#Funny
#Beautiful
#Ingenious
#Courageous	
#Longwinded	
#Confusing	
#Informative	
#Fascinating	
#Unconvincing
#Persuasive	
#Jaw-dropping
#OK	
#Obnoxious	
#Inspiring	

#tags	

#views

#라벨 : comments, 피처 : duration, views, comments, funny,

(2550, 19)

In [3]:
def preprocess_features(data):
    features = data[
        ['duration', 'views']
    ]
    processed_features = features.copy()
    processed_features['emotions'] = np.log(data['Funny'] + data['Beautiful'] + data['Ingenious'] + data['Courageous'] + data['Longwinded'] + data['Confusing'] + data['Informative'] + data['Fascinating'] + data['Unconvincing'] + data['Persuasive'] + data['Jaw-dropping'] + data['OK'] + data['Obnoxious'] + data['Inspiring'])
    return processed_features

def preprocess_targets(data):
    output = pd.DataFrame()
    output['comments'] = data['comments']
    return output

In [4]:
training_examples = preprocess_features(data.head(2000).head(1600))
training_targets = preprocess_targets(data.head(2000).head(1600))

validation_examples = preprocess_features(data.head(2000).tail(400))
validation_targets = preprocess_targets(data.head(2000).tail(400))

test_examples = preprocess_features(data.tail(550))
test_targets = preprocess_targets(data.tail(550))



In [5]:
print('Examples')
display.display(training_examples.describe())
display.display(validation_examples.describe())
display.display(test_examples.describe())

Examples


,duration,views,emotions
count,1600.000,1.600000e+03,1600.000000
mean,829.770,1.739469e+06,7.344655
std,383.573,2.845112e+06,0.866941
min,135.000,6.666800e+04,4.219508
25%,581.500,7.545185e+05,6.765904
50%,847.000,1.118001e+06,7.274479
75%,1050.250,1.677199e+06,7.808526
max,5256.000,4.722711e+07,11.449453


,duration,views,emotions
count,400.000000,4.000000e+02,400.000000
mean,797.452500,1.629891e+06,7.342244
std,364.769413,1.768267e+06,0.844199
min,163.000000,1.153460e+05,4.330733
25%,528.500000,7.561098e+05,6.784457
50%,833.500000,1.198081e+06,7.302817
75%,1026.250000,1.784103e+06,7.818731
max,3608.000000,1.686158e+07,10.549491


,duration,views,emotions
count,550.000000,5.500000e+02,550.000000
mean,838.160000,1.628275e+06,7.340894
std,351.380991,1.766749e+06,0.876141
min,153.000000,5.044300e+04,4.584967
25%,605.250000,7.618972e+05,6.770782
50%,869.500000,1.113544e+06,7.275172
75%,1055.500000,1.711005e+06,7.885423
max,3573.000000,1.883098e+07,10.026369


In [6]:
def construct_feature_columns(input_feautures):
    return set([tf.feature_column.numeric_column(feature) for feature in input_features])

In [7]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    features = {key:np.array(value) for key,value in dict(features).items()}
    ds = Dataset.from_tensor_slices((features, targets))
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    if shuffle:
        ds = ds.shuffle(10000)
    
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

In [8]:
def train_nn_regression_model(steps, batch_size, hidden_units, training_examples, training_targets, validation_examples, validation_targets):
    periods = 10
    steps_per_period = steps/ periods

    dnn_regressor = tf.estimator.DNNRegressor(
    feature_columns = construct_feature_columns(training_examples), hidden_units=hidden_units, optimizer = lambda: tf.keras.optimizers.Adam(learning_rate=0.1,
            global_step=tf.compat.v1.train.get_global_step(),
            decay_steps=10000,
            decay_rate=0.96))
    
    training_input_fn = lambda : my_input_fn(training_examples,
                                            training_targets["comments"], batch_size = batch_size)
    predict_training_input_fn = lambda : my_input_fn(training_examples, training_targets['comments'],
                                                    num_epochs = 1, shuffle=False)
    predict_validation_input_fn = lambda : my_input_fn(validation_examples,
                                                      validation_targets['comments'], num_epochs=1, shuffle=False)
    
    print('Training model...')
    print('RMSE on training data : ')
    training_rmse = []
    validation_rmse = []
    for period in range(0, periods):
        dnn_regressor.train(
        input_fn = training_input_fn,
        steps = steps_per_period)
        
        training_predictions = dnn_regressor.predict(input_fn=predict_training_input_fn)
        training_predictions = np.array([item['predictions'][0] for item in training_predictions])
        
        validation_predictions = dnn_regressor.predict(input_fn=predict_validation_input_fn)
        validation_predictions = np.array([item['predictions'][0] for item in validation_predictions])
        
        training_rmse_ = math.sqrt(
        metrics.mean_squared_error(training_predictions, training_targets))
        validation_rmse_ = math.sqrt(
        metrics.mean_squared_error(validation_predictions, validation_targets))
        training_rmse.append(training_rmse_)
        validation_rmse.append(validation_rmse_)
    print("Model training finished.")
    
    plt.ylabel('RMSE')
    plt.xlabel("Periods")
    plt.title("RMSE Over Periods")
    plt.tight_layout()
    plt.plot(training_rmse, label='training')
    plt.plot(validation_rmse, label='validation')
    plt.legend()
    
    print('Final RMSE on training data : %0.4f' % training_rmse_)
    print('Final RMSE on validation data : %0.4f ' % validation_rmse_)
    
    return dnn_regressor, training_rmse, validation_rmse

In [16]:
_ = train_nn_regression_model(steps=5000,
    batch_size=70,
    hidden_units=[10, 10],
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

NameError: name 'input_features' is not defined

In [ ]:
_ = train_nn_regression_model(
    my_optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.0007),
    steps=5000,
    batch_size=70,
    hidden_units=[10, 10],
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)